In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/zinc250k/250k_rndm_zinc_drugs_clean_3.csv


In [2]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 52.3 MB/s eta 0:00:00:00:0100:01


In [3]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [4]:
# Load the dataset and select a subset of 10,000 rows
df = pd.read_csv('/kaggle/input/zinc250k/250k_rndm_zinc_drugs_clean_3.csv')  # Adjust path to your dataset
df = df.head(50000)  # Select the first 10,000 rows

In [5]:
df

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182
...,...,...,...,...
49995,CCCCN(C(=O)N[C@@H](C)c1nc(C)cs1)[C@H](C)c1cccc...,5.08542,0.729332,3.017260
49996,CCc1ccc(O)c(C(=O)N2CCN(Cc3nc(-c4cccc(C(F)(F)F)...,3.98140,0.598725,2.372689
49997,CCn1c(SCC(=O)Nc2c(C#N)c(C)c(C)n2C[C@H]2CCCO2)n...,4.25602,0.535909,2.988292
49998,CC1CC[NH+](CCCNC(=O)NC[C@H]2CCCN(c3ncccn3)C2)C...,0.69710,0.624215,3.908818


In [6]:
import torch
# Set device to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [7]:
# Function to convert SMILES to molecular descriptors (features)
def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    descriptors = [
        Descriptors.MolWt(mol),          # Molecular Weight
        Descriptors.MolLogP(mol),        # LogP
        Descriptors.NumHDonors(mol),     # Number of Hydrogen Donors
        Descriptors.NumHAcceptors(mol),  # Number of Hydrogen Acceptors
        Descriptors.TPSA(mol),           # Topological Polar Surface Area
        Descriptors.NumRotatableBonds(mol)  # Number of Rotatable Bonds
    ]
    return descriptors

In [8]:

# Compute molecular descriptors for each SMILES string
df['descriptors'] = df['smiles'].apply(compute_descriptors)

In [9]:
df

,smiles,logP,qed,SAS,descriptors
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095,"[325.38300000000004, 5.050600000000004, 1, 2, ..."
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004,"[285.39500000000004, 3.1137000000000015, 1, 5,..."
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633,"[382.46299999999997, 4.967780000000004, 0, 3, ..."
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753,"[409.5300000000003, 4.000220000000003, 0, 5, 6..."
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182,"[413.9100000000003, 3.6095600000000028, 1, 6, ..."
...,...,...,...,...,...
49995,CCCCN(C(=O)N[C@@H](C)c1nc(C)cs1)[C@H](C)c1cccc...,5.08542,0.729332,3.017260,"[345.51200000000006, 5.0854200000000045, 1, 3,..."
49996,CCc1ccc(O)c(C(=O)N2CCN(Cc3nc(-c4cccc(C(F)(F)F)...,3.98140,0.598725,2.372689,"[460.45600000000013, 3.9814000000000034, 1, 6,..."
49997,CCn1c(SCC(=O)Nc2c(C#N)c(C)c(C)n2C[C@H]2CCCO2)n...,4.25602,0.535909,2.988292,"[437.56900000000024, 4.256020000000004, 1, 7, ..."
49998,CC1CC[NH+](CCCNC(=O)NC[C@H]2CCCN(c3ncccn3)C2)C...,0.69710,0.624215,3.908818,"[375.5410000000001, 0.6971000000000018, 3, 4, ..."


In [10]:
# Drop rows where descriptors couldn't be calculated
df = df.dropna(subset=['descriptors'])

# Prepare the feature matrix (X) and target variables (y_logP, y_qed, y_sas)
X = np.array(df['descriptors'].tolist())  # Descriptors as features
y_logP = df['logP'].values
y_qed = df['qed'].values
y_sas = df['SAS'].values

# Split data into train and test sets for each target property
X_train_logP, X_test_logP, y_train_logP, y_test_logP = train_test_split(X, y_logP, test_size=0.2, random_state=42)
X_train_qed, X_test_qed, y_train_qed, y_test_qed = train_test_split(X, y_qed, test_size=0.2, random_state=42)
X_train_sas, X_test_sas, y_train_sas, y_test_sas = train_test_split(X, y_sas, test_size=0.2, random_state=42)

# Train separate Random Forest models for each property
rf_logP = RandomForestRegressor(n_estimators=100, random_state=42)
rf_qed = RandomForestRegressor(n_estimators=100, random_state=42)
rf_sas = RandomForestRegressor(n_estimators=100, random_state=42)

rf_logP.fit(X_train_logP, y_train_logP)
rf_qed.fit(X_train_qed, y_train_qed)
rf_sas.fit(X_train_sas, y_train_sas)

# Make predictions
y_pred_logP = rf_logP.predict(X_test_logP)
y_pred_qed = rf_qed.predict(X_test_qed)
y_pred_sas = rf_sas.predict(X_test_sas)

# Define a function to calculate MAPE
def mean_absolute_percentage_error(y_true, y_pred):
    non_zero_indices = y_true != 0
    if np.any(non_zero_indices):
        mape = np.mean(np.abs((y_true[non_zero_indices] - y_pred[non_zero_indices]) / y_true[non_zero_indices])) * 100
    else:
        mape = np.nan
    return mape

# Evaluate model performance for logP
mse_logP = mean_squared_error(y_test_logP, y_pred_logP)
rmse_logP = np.sqrt(mse_logP)
mae_logP = mean_absolute_error(y_test_logP, y_pred_logP)
r2_logP = r2_score(y_test_logP, y_pred_logP)
mape_logP = mean_absolute_percentage_error(y_test_logP, y_pred_logP)

# Evaluate model performance for QED
mse_qed = mean_squared_error(y_test_qed, y_pred_qed)
rmse_qed = np.sqrt(mse_qed)
mae_qed = mean_absolute_error(y_test_qed, y_pred_qed)
r2_qed = r2_score(y_test_qed, y_pred_qed)
mape_qed = mean_absolute_percentage_error(y_test_qed, y_pred_qed)

# Evaluate model performance for SAS
mse_sas = mean_squared_error(y_test_sas, y_pred_sas)
rmse_sas = np.sqrt(mse_sas)
mae_sas = mean_absolute_error(y_test_sas, y_pred_sas)
r2_sas = r2_score(y_test_sas, y_pred_sas)
mape_sas = mean_absolute_percentage_error(y_test_sas, y_pred_sas)

# Print the results
print(f"Random Forest for logP: MSE={mse_logP:.4f}, RMSE={rmse_logP:.4f}, MAE={mae_logP:.4f}, R²={r2_logP:.4f}, MAPE={mape_logP:.2f}%")
print(f"Random Forest for QED: MSE={mse_qed:.4f}, RMSE={rmse_qed:.4f}, MAE={mae_qed:.4f}, R²={r2_qed:.4f}, MAPE={mape_qed:.2f}%")
print(f"Random Forest for SAS: MSE={mse_sas:.4f}, RMSE={rmse_sas:.4f}, MAE={mae_sas:.4f}, R²={r2_sas:.4f}, MAPE={mape_sas:.2f}%")


Random Forest for logP: MSE=0.0001, RMSE=0.0083, MAE=0.0003, R²=1.0000, MAPE=0.03%
Random Forest for QED: MSE=0.0077, RMSE=0.0877, MAE=0.0638, R²=0.6116, MAPE=10.29%
Random Forest for SAS: MSE=0.3307, RMSE=0.5751, MAE=0.4500, R²=0.5272, MAPE=15.70%
